# Make Calibration WAV Files
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y %Z", localtime (time())))
%plot --format svg
# format long

Last updated: Wednesday  1 January 2020 UTC

In [ ]:
pkg install -forge struct
pkg install -forge io
pkg install -forge statistics
pkg install -forge optim

In [ ]:
fcLow = 200;
fcHigh = 2600;

In [ ]:
fcMid = sqrt(fcHigh/fcLow) * fcLow

In [ ]:
F = fcLow:1:fcHigh;
columns(F)
sqrt(fcLow*fcHigh)

In [ ]:
J = log(F);
jmin = min(J)
jmax = max(J)
jmid = mean([jmin, jmax])
H = abs(J - jmid);
hmin = min(H)
hmax = max(H)

In [ ]:
# What are the factors of the number of frequencies?
Factors = factor(columns(F)-1);
# Determine the possible intervals by looking at all products of choose
Possible_Intervals = unique(cell2mat(arrayfun(@(t) unique(prod(nchoosek(Factors, t), 2))', [1:columns(Factors)], "UniformOutput", false)))
# Then pick one of them
intervals = Possible_Intervals(9)

Sort F so that it is in the order log distance from fcMid, and also compute what interval they are in.

In [ ]:
G = sortrows(horzcat(H', ceil(intervals*H'/hmax), F'), [1]);

Sample a number of rows

In [ ]:
format long g
Sample = vertcat(G(1:floor(end/intervals):end,:), G(rows(G),:))
format

In [ ]:
G(find(G(:,3)==721),:)

In [ ]:
sum(G(:,2) == 1)
sum(G(:,2) == 1:1:intervals)

Ensure that there are no values of H that are duplicated, meaning that there are a pair of frequencies, one each on each side of fcMid, that are equidistant log-wise, from fcMid. If there are, then we would have to be careful how we subdivide F later on. If there is at least one, then the following result will be zero.

In [ ]:
sort(abs(diff(H)))(1)

Helps that fcMid is not integral, like members of F.

In [ ]:
TestFreq = G(:,3);
iter = 0;
filename = sprintf("TestFreq.mat", iter);
save(filename,"TestFreq");
# save "TestFreq.mat" TestFreq;

In [ ]:
sampleRate = 16000;
numPeriods = 10;
period = numPeriods;
Tshort = 0:1/sampleRate:1;
T = 0:1/sampleRate:period;

Ensure that you run `Nonlin_min_Install` notebook to install this package.

In [ ]:
pkg load optim

In [ ]:
# TestPhi is a column vector, 0 <= phi <= pi, length m
# TestFreq is a column vector, length m
# Sig is a row vector
# Tshort is a row vector
function cost = costAny(Phi_trial, CosTerm, Sig)
  cost = max(abs(sum(cos(CosTerm + Phi_trial)) + Sig));
endfunction

In [ ]:
tic = time();
TestFreq = [ G(1,3) ];
TestPhi = [ 0 ];
cosArg = 2*pi*TestFreq * Tshort + TestPhi;
Sig = cos(cosArg);
size(Sig)
max(abs(Sig))
printf("toc: %d seconds\n", time() - tic)
numIndexes = 16;
iter=1;
for firstIndex = 2:numIndexes:rows(G)
  printf("iter: %d\n", iter);
  Sigfilename = sprintf("Sig_%d.mat", iter);
  if (2 != exist (Sigfilename, "file"))
    TestFreq = G(firstIndex:firstIndex+numIndexes-1, 3);
    CosTerm = 2*pi*TestFreq * Tshort;
    tic = time();
  
    init_p = pi*rand(rows(TestFreq),1);
    lbound = zeros(rows(TestFreq), 1);
    ubound = pi .* ones(rows(TestFreq), 1);
    [TestPhi, objf, cvg] = nonlin_min (@ (Phi_trial) costAny(Phi_trial, CosTerm, Sig), init_p, optimset ("Algorithm", "samin", 
      "lbound", lbound,
      "ubound", ubound,
      "Display", "iter"
    ));
    printf("toc: %d seconds\n", time() - tic)
    cosArg = 2*pi*TestFreq * Tshort + TestPhi;
    Sig = sum(cos(cosArg)) + Sig;
    save(Sigfilename, "Sig");
    filename = sprintf("TestFreq_%d.mat", iter);
    save(filename, "TestFreq");
    filename = sprintf("TestPhi_%d.mat", iter);
    save(filename, "TestPhi");
  else
    load(Sigfilename)
  endif
  
  max(abs(Sig))
  iter++;
endfor

In [ ]:
Sig(1)
Sig(end)
# Repeat Sig to expand it to numPeriods
Sig = [(Sig(1:end-1)' * ones(1, numPeriods))(:)', Sig(1)];

In [ ]:
plot(T, Sig)

In [ ]:
RC = (1 - cos(2*pi*T/period))/2;

In [ ]:
E_cos_pulse = fftshift(fft(RC .* Sig, sampleRate*period*10 + 1));
# E_cos_pulse = fftshift(fft(RC .* Sig));
F_base = linspace(-sampleRate/2, sampleRate/2, columns(E_cos_pulse));

In [ ]:
plot(F_base, abs(E_cos_pulse))
grid on
grid minor
# axis([fcLow-10 fcLow+10], "tic")